In [ ]:
drive_folder = "Machine_Unlearning_Drive/NLPResults/"

ssd_folder = "SSD/"

scrub_folder = "SCRUB/"

github_folder = "Machine_Unlearning/"

!pip install datasets transformers scikit-learn torch torchvision seaborn matplotlib

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

from datasets import load_dataset
import random
from sklearn import metrics, model_selection, preprocessing
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
import json
import copy

import transformers
from transformers import AdamW, get_linear_schedule_with_warmup

from Machine_Unlearning.Metrics.metrics import *

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print("Running on device:", DEVICE.upper())

def seed_everything(seed):
  RNG = torch.Generator().manual_seed(seed)
  torch.manual_seed(seed)
  random.seed(seed)
  np.random.seed(seed)
  return RNG

SEED = 44
SPLIT = 0.3
RNG = seed_everything(SEED)
results = {}

In [ ]:
go_emotions = load_dataset("go_emotions")
data = go_emotions.data

train = data["train"].to_pandas()
valid = data["validation"].to_pandas()
test = data["test"].to_pandas()

print(train.shape, valid.shape, test.shape)
# (43410, 3) (5426, 3) (5427, 3)

train.head()

In [ ]:
n_labels = 28
def one_hot_encoder(df):
    one_hot_encoding = []
    for i in range(len(df)):
        temp = [0] * n_labels
        label_indices = df.iloc[i]["labels"]
        for index in label_indices:
            temp[index] = 1
        one_hot_encoding.append(temp)

    return pd.DataFrame(one_hot_encoding)

train_ohe_labels = one_hot_encoder(train)
valid_ohe_labels = one_hot_encoder(valid)
test_ohe_labels = one_hot_encoder(test)

print(train_ohe_labels.shape)
#(43410, 28)

train = pd.concat([train, train_ohe_labels], axis=1)
valid = pd.concat([valid, valid_ohe_labels], axis=1)
test = pd.concat([test, test_ohe_labels], axis=1)

train.head()

In [ ]:
class GoEmotionDataset:
    def __init__(self, texts, labels, tokenizer, max_len):
        self.texts = texts
        self.labels = labels

        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, index):
        text = self.texts[index]
        label = self.labels[index]

        inputs = self.tokenizer.__call__(text,
                                        None,
                                        add_special_tokens=True,
                                        max_length=self.max_len,
                                        padding="max_length",
                                        truncation=True,
                                        )
        ids = inputs["input_ids"]
        mask = inputs["attention_mask"]

        return {
            "ids": torch.tensor(ids, dtype=torch.long),
            "mask": torch.tensor(mask, dtype=torch.long),
            "labels": torch.tensor(label, dtype=torch.long)
        }

In [ ]:
class GoEmotionClassifier(nn.Module):
    def __init__(self, n_train_steps, n_classes, do_prob, bert_model):
        super(GoEmotionClassifier, self).__init__()
        self.bert = bert_model
        self.dropout = nn.Dropout(do_prob)
        self.out = nn.Linear(768, n_classes)
        self.n_train_steps = n_train_steps
        self.step_scheduler_after = "batch"

    def forward(self, ids, mask):
        output_1 = self.bert(ids, attention_mask=mask)["pooler_output"]
        output_2 = self.dropout(output_1)
        output = self.out(output_2)
        return output

In [ ]:
def train_fn(data_loader, model, optimizer, device, scheduler):
    '''
        Modified from Abhishek Thakur's BERT example:
        https://github.com/abhishekkrthakur/bert-sentiment/blob/master/src/engine.py
    '''

    train_loss = 0.0
    model.train()
    for bi, d in enumerate(data_loader):
        ids = d["ids"]
        mask = d["mask"]
        targets = d["labels"]

        ids = ids.to(device, dtype=torch.long)
        mask = mask.to(device, dtype=torch.long)
        targets = targets.to(device, dtype=torch.float)

        optimizer.zero_grad()
        outputs = model(ids=ids, mask=mask)

        loss = nn.BCEWithLogitsLoss()(outputs, targets.float())
        loss.backward()
        train_loss += loss.item()
        optimizer.step()
    return train_loss


def eval_fn(data_loader, model, device):
    '''
        Modified from Abhishek Thakur's BERT example:
        https://github.com/abhishekkrthakur/bert-sentiment/blob/master/src/engine.py
    '''
    eval_loss = 0.0
    model.eval()
    fin_targets = []
    fin_outputs = []
    with torch.no_grad():
        for bi, d in enumerate(data_loader):
            ids = d["ids"]
            mask = d["mask"]
            targets = d["labels"]

            ids = ids.to(device, dtype=torch.long)
            mask = mask.to(device, dtype=torch.long)
            targets = targets.to(device, dtype=torch.float)

            outputs = model(ids=ids, mask=mask)
            loss = nn.BCEWithLogitsLoss()(outputs, targets.float())
            eval_loss += loss.item()
            fin_targets.extend(targets)
            fin_outputs.extend(torch.sigmoid(outputs))
    return eval_loss, fin_outputs, fin_targets

In [ ]:
tokenizer = transformers.RobertaTokenizer.from_pretrained("roberta-base", do_lower_case=True)
bert_model = transformers.RobertaModel.from_pretrained("roberta-base")

def build_dataset(tokenizer_max_len):
    train_dataset = GoEmotionDataset(train.text.tolist(), train[range(n_labels)].values.tolist(), tokenizer, tokenizer_max_len)
    valid_dataset = GoEmotionDataset(valid.text.tolist(), valid[range(n_labels)].values.tolist(), tokenizer, tokenizer_max_len)
    test_dataset = GoEmotionDataset(test.text.tolist(), test[range(n_labels)].values.tolist(), tokenizer, tokenizer_max_len)
    return train_dataset, valid_dataset, test_dataset

def build_dataloader(train_dataset, valid_dataset, test_dataset,  batch_size):
    train_data_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=1)
    valid_data_loader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=True, num_workers=1)
    test_data_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True, num_workers=1)

    return train_data_loader, valid_data_loader, test_data_loader

def ret_model(n_train_steps, do_prob):
  model = GoEmotionClassifier(n_train_steps, n_labels, do_prob, bert_model=bert_model)
  return model

In [ ]:
def accuracy(net, loader):
    """Return accuracy on a dataset given by the data loader."""
    correct = 0
    total = 0
    with torch.no_grad():
            for bi, d in enumerate(loader):
                ids = d["ids"]
                mask = d["mask"]
                targets = d["labels"]

                ids = ids.to(device, dtype=torch.long)
                mask = mask.to(device, dtype=torch.long)
                targets = targets.to(device, dtype=torch.float)


                outputs = net(ids=ids, mask=mask)

                correct_vec = (torch.argmax(targets,dim=1) == torch.argmax(outputs,dim=1))
                correct += correct_vec.sum().item()
                total += int(targets.size()[0])

    return correct / total

In [ ]:
def compute_entropies(net, loader):
    """Auxiliary function to compute per-sample losses"""

    #data_loader = torch.utils.data.DataLoader(loader.dataset, batch_size=1, shuffle=False, num_workers = 2, prefetch_factor = 10)
    prob = []
    DEVICE = next(net.parameters()).device
    with torch.no_grad():
        for d in loader:
            #batch = [tensor.to(next(model.parameters()).device) for tensor in batch]
            ids = d["ids"]
            mask = d["mask"]
            targets = d["labels"]

            ids = ids.to(device, dtype=torch.long)
            mask = mask.to(device, dtype=torch.long)
            targets = targets.to(device, dtype=torch.float)


            outputs = net(ids=ids, mask=mask)
            prob.append(torch.nn.functional.softmax(outputs, dim=-1).data)

    p = torch.cat(prob)
    return (-torch.where(p > 0, p * p.log(), p.new([0.0])).sum(dim=-1, keepdim=False)).numpy(force=True)

def compute_losses(net, loader):
    """Auxiliary function to compute per-sample losses"""

    DEVICE = next(net.parameters()).device
    criterion = nn.BCEWithLogitsLoss(reduction='none')
    all_losses = []

    for d in loader:
        ids = d["ids"]
        mask = d["mask"]
        targets = d["labels"]

        ids = ids.to(device, dtype=torch.long)
        mask = mask.to(device, dtype=torch.long)
        targets = targets.to(device, dtype=torch.float)


        logits = net(ids=ids, mask=mask)
        losses = criterion(logits, targets).sum(dim=1).numpy(force=True)
        #print(losses)
        for l in losses:
            all_losses.append(l)

    return np.array(all_losses)

In [ ]:
def readout(model,name):
  RNG = seed_everything(SEED)
  test_entropies = compute_entropies(model, test_data_loader)
  retain_entropies = compute_entropies(model, retain_loader)
  forget_entropies = compute_entropies(model, forget_loader)


  results[f"test_entropies_{name}"] = test_entropies.tolist()
  results[f"retain_entropies_{name}"] = retain_entropies.tolist()
  results[f"forget_entropies_{name}"] = forget_entropies.tolist()

  test_losses = compute_losses(model, test_data_loader)
  retain_losses = compute_losses(model, retain_loader)
  forget_losses = compute_losses(model, forget_loader)

  results[f"test_losses_{name}"] = test_losses.tolist()
  results[f"retain_losses_{name}"] = retain_losses.tolist()
  results[f"forget_losses_{name}"] = forget_losses.tolist()

  # Since we have more forget losses than test losses, sub-sample them, to have a class-balanced dataset.
  gen = np.random.default_rng(1)
  if len(test_losses) > len(forget_losses):
    gen.shuffle(test_losses)
    test_losses = test_losses[: len(forget_losses)]
  else:
    gen.shuffle(forget_losses)
    forget_losses = forget_losses[: len(test_losses)]
    # make sure we have a balanced dataset for the MIA
  assert len(test_losses) == len(forget_losses)

  samples_mia = np.concatenate((test_losses, forget_losses)).reshape((-1, 1))
  labels_mia = [0] * len(test_losses) + [1] * len(forget_losses)

  mia_scores = simple_mia(samples_mia, labels_mia)

  print(
      f"The MIA has an accuracy of {mia_scores.mean():.3f} on forgotten vs unseen images"
  )

  results[f"MIA_losses_{name}"] = mia_scores.mean()

  gen = np.random.default_rng(1)
  if len(test_entropies) > len(forget_entropies):
    gen.shuffle(test_entropies)
    test_entropies = test_entropies[: len(forget_entropies)]
  else:
    gen.shuffle(forget_entropies)
    forget_entropies = forget_entropies[: len(test_entropies)]
    # make sure we have a balanced dataset for the MIA
  assert len(test_entropies) == len(forget_entropies)

  samples_mia = np.concatenate((test_entropies, forget_entropies)).reshape((-1, 1))
  labels_mia = [0] * len(test_entropies) + [1] * len(forget_entropies)

  mia_scores = simple_mia(samples_mia, labels_mia)

  print(
      f"The MIA has an accuracy of {mia_scores.mean():.3f} on forgotten vs unseen images"
  )

  results[f"MIA_entropies_{name}"] = mia_scores.mean()

  results[f"train_accuracy_{name}"] = accuracy(model, retain_loader)
  results[f"test_accuracy_{name}"] = accuracy(model, test_data_loader)
  results[f"forget_accuracy_{name}"] = accuracy(model, forget_loader)

  print("Train acc:"+ str(results[f"train_accuracy_{name}"]))
  print("Test acc:"+ str(results[f"test_accuracy_{name}"]))
  print("Forget acc:" +str(results[f"forget_accuracy_{name}"]))

In [ ]:
config = {"tokenizer_max_len" : 40,
          "batch_size": 128,
          "dropout": 0.5,
          "epochs":10}

train_dataset, valid_dataset, test_dataset = build_dataset(config['tokenizer_max_len'])
train_data_loader, valid_data_loader, test_data_loader = build_dataloader(train_dataset, valid_dataset, test_dataset, config['batch_size'])
print("Length of Train Dataloader: ", len(train_data_loader))
print("Length of Valid Dataloader: ", len(valid_data_loader))
GEN1 = torch.Generator().manual_seed(42)
retain_set, forget_set = torch.utils.data.random_split(train_dataset,[1-SPLIT,SPLIT],GEN1)

forget_loader = torch.utils.data.DataLoader(
    forget_set, batch_size=128, shuffle=True, num_workers=2 , generator=RNG
)
retain_loader = torch.utils.data.DataLoader(
    retain_set, batch_size=128, shuffle=True, num_workers=2, generator=RNG
)
print("Length of Train Dataloader: ", len(retain_loader))
print("Length of Valid Dataloader: ", len(valid_data_loader))

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

n_train_steps = int(len(retain_set) / config['batch_size'] * 10)

In [ ]:
# load model with pre-trained weights
model = ret_model(n_train_steps, config['dropout'])

optimizer = AdamW(model.parameters(), lr=5e-05)
scheduler = torch.optim.lr_scheduler.LinearLR(optimizer, start_factor=1.0, end_factor=0.1, total_iters=config['epochs'])
model.to(device)
#model = nn.DataParallel(model) #READ bout that

n_epochs = config['epochs']

best_val_loss = 100
for epoch in range(n_epochs):
    train_loss = train_fn(retain_loader, model, optimizer, device, scheduler)

    eval_loss, preds, labels = eval_fn(valid_data_loader, model, device)

    avg_train_loss, avg_val_loss = train_loss / len(retain_loader), eval_loss / len(valid_data_loader)

    print("Average Train loss: ", avg_train_loss)
    print("Average Valid loss: ", avg_val_loss)

    scheduler.step()

In [ ]:
readout(model,"retrained")

In [ ]:
with open(drive_folder+f"results_NLP_SPLIT_{int(SPLIT*100)}%_SEED_{SEED}_retrained.json", 'w') as fout:
  json.dump(results, fout)